In [0]:
import numpy as np
import skimage as sm
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [0]:
orange = cv2.imread(filename="orange.jpg")
apple = cv2.imread(filename="apple.jpg")
bnn = cv2.imread(filename="banana.jpg")

In [0]:
fruits = ["apple", "orange", "banana"]
fruits_data = [apple, orange, bnn]

In [0]:
idx = 0
for fruit in fruits_data:
  gray = cv2.cvtColor(fruit, cv2.COLOR_BGR2GRAY)
  th, threshed = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
  morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
  #cv2_imshow(morphed)
  cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
  cnt = sorted(cnts, key=cv2.contourArea)[-1]
  x,y,w,h = cv2.boundingRect(cnt)
  dst = fruit[y:y+h, x:x+w]
  name = fruits[idx]
  cv2.imwrite(name+"-cropped.jpg",dst)
  idx+=1

In [0]:
orange_c = cv2.imread(filename="orange-cropped.jpg")
apple_c = cv2.imread(filename="apple-cropped.jpg")
bnn_c = cv2.imread(filename="banana-cropped.jpg")

In [0]:
orange_mean1 = np.mean(orange/256)
apple_mean1 = np.mean(apple/256)
bnn_mean1 = np.mean(bnn/256)
orange_mean2 = np.mean(orange_c/256)
apple_mean2 = np.mean(apple_c/256)
bnn_mean2 = np.mean(bnn_c/256)

In [0]:
print("Uncropped mean RGB Values:",orange_mean1, apple_mean1,bnn_mean1)
print("Cropped mean RGB Values:",orange_mean2, apple_mean2,bnn_mean2)

Uncropped mean RGB Values: 0.8891104435750952 0.7164393777967185 0.9108639915854402
Cropped mean RGB Values: 0.6745593403774112 0.6276784301633728 0.8148599006549276


# Blob Detection

In [0]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Change thresholds
params.minThreshold = 0;
params.maxThreshold = 240;
 
# Filter by Area.
params.filterByArea = True
params.minArea = 10000
# Filter by Circularity
params.filterByCircularity = False

 
# Filter by Convexity
params.filterByConvexity = False

# Filter by Inertia
params.filterByInertia = False


In [0]:
fruits_c = ["apple-cropped.jpg", "orange-cropped.jpg", "banana-cropped.jpg"]
fruits_cropped = [apple_c, orange_c, bnn_c]
for fruit in fruits_c:
  im = cv2.imread(fruit, cv2.IMREAD_GRAYSCALE)
  ret,thresh = cv2.threshold(im,240,255,cv2.THRESH_BINARY_INV)
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
  thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
  # Set up the detector with default parameters.
  detector = cv2.SimpleBlobDetector_create(params)
  # Detect blobs.
  keypoints = detector.detect(thresh)
  label, N = sm.measure.label(thresh, background=0, return_num=True)
  reg = sm.measure.regionprops(label,thresh)
  print(N)
  area = reg[0].area
  #print(area)
  conv_area = reg[0].convex_area
  perimeter = reg[0].perimeter
  inertia_a = reg[0].inertia_tensor_eigvals[0]
  inertia_b = reg[0].inertia_tensor_eigvals[1]
  convexity = area / conv_area
  circularity = (4* np.pi * area) / (perimeter**2)
  inertia = inertia_b/inertia_a
  print(convexity,circularity,inertia)

1
0.9852368679147416 0.8255264501896089 0.9072711371760853
1
0.9350518433179723 0.6841935757303954 0.745505492773831
1
0.5804595644159702 0.2771780580295129 0.15613543041649006


/usr/local/lib/python3.6/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.6/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
